# Cryptocurrency trading bot.

In [1]:
##%pip install python-binance==0.7.5
#%pip install python-binance

In [2]:
!python bootstrap.py

100%|████████████████████████████████████| 1426/1426 [18:37<00:00,  1.28 pair/s]
1426 named pair [02:33,  9.31 named pair/s]
100%|████████████████████████████████████| 1426/1426 [08:32<00:00,  2.78 pair/s]
1426 named pair [00:10, 135.66 named pair/s]
100%|██████████████████████████████████████████| 2/2 [00:04<00:00,  2.08s/ pair]
Bootstrapping done!
